In [1]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

pd.set_option('display.max_columns', None)

import xgboost as xgb
import json
from typing import Dict
from shapely import wkt, ops
from pyproj import Transformer
from tqdm import tqdm


## Prepare dataset
### Download and extract data

We are focusing on Paris for now.

In [ ]:
url = "https://open-data.s3.fr-par.scw.cloud/bdnb_millesime_2022-10-d/millesime_2022-10-d_dep75/open_data_millesime_2022-10-d_dep75_csv.zip"

os.system(f"wget -P data {url}")
os.system("unzip -o data/open_data_millesime_2022-10-d_dep75_csv.zip -d data")
os.system("rm data/open_data_millesime_2022-10-d_dep75_csv.zip")

### Extract geometries and DPE as a GeoJSON

In [2]:
df_geom = pd.read_csv("data/csv/batiment_groupe.csv", sep=",")
df_geom = df_geom[["batiment_groupe_id", "geom_groupe"]]
df_geom.shape

(71458, 2)

In [3]:
df_dpe = pd.read_csv("data/csv/batiment_groupe_dpe.csv", sep=",")
df_dpe = df_dpe[["batiment_groupe_id", "class_conso_ener_mean"]]
df_dpe.shape

(47087, 2)

In [4]:
df_height = pd.read_csv("data/csv/batiment_groupe_bdtopo_bat.csv", sep=",")
df_height = df_height[["batiment_groupe_id", "hauteur_mean"]]
df_height.shape

(71458, 2)

In [8]:
# merge 3 dataframes
df_mapbox = df_geom.merge(df_dpe, on="batiment_groupe_id", how="left")
df_mapbox = df_mapbox.merge(df_height, on="batiment_groupe_id", how="left")
df_mapbox.shape

(71458, 4)

In [9]:
df_mapbox = df_mapbox.fillna("N")
df_mapbox.head()

,batiment_groupe_id,geom_groupe,class_conso_ener_mean,hauteur_mean
0,75102000AM0186_b9ea9d805215efe,"MULTIPOLYGON (((652200.8 6863089.2,652200.0 68...",N,20
1,75105000AI0013_15713814b72666b,"MULTIPOLYGON (((652253.9 6861094.1,652257.1 68...",N,5
2,uf751170005758_eb9ba44d60e1c89,MULTIPOLYGON (((649028.813311109 6866141.65815...,N,5
3,75114000AN0113_2e395dd3c45f67d,MULTIPOLYGON (((650650.343315098 6859622.70889...,N,15
4,75111000CL0129_4fa62d2e1d35454,"MULTIPOLYGON (((655479.2 6861827.3,655476.9 68...",N,17


In [6]:
def df_to_geojson(df: pd.DataFrame, path: str, batch_size=500):
    transformer = Transformer.from_crs("epsg:2154", "epsg:4326", always_xy=True)
    
    with open(path, 'w') as f:
        f.write('{"type": "FeatureCollection", "features": [')
        
        first_feature = True
        for i in tqdm(range(0, len(df), batch_size), desc="Processing batches"):
            batch = df.iloc[i:i+batch_size]
            
            for idx, row in batch.iterrows():  # idx captures the index position of the row
                try:
                    geometry_wkt = row["geom_groupe"] if pd.notna(row["geom_groupe"]) else None
                    if geometry_wkt:
                        shape = wkt.loads(geometry_wkt)
                        shape = ops.transform(transformer.transform, shape)
                        geometry = shape.__geo_interface__
                    else:
                        geometry = None
                except Exception as e:
                    print(f"Error with {row['batiment_groupe_id']}")
                    print(e)
                    continue
                
                if geometry:
                    feature = {
                        "type": "Feature",
                        "id": idx,  # Using the row index as the feature ID
                        "properties": {prop: row[prop] for prop in df.columns if prop != "geom_groupe"},
                        "geometry": geometry,
                    }
                    
                    if not first_feature:
                        f.write(',')
                    else:
                        first_feature = False
                    
                    json.dump(feature, f)
                    
        f.write(']}')

In [10]:
if not os.path.exists("data/geojson"):
    os.makedirs("data/geojson")
df_to_geojson(df_mapbox, "data/geojson/dpe_initial.geojson")

Processing batches: 100%|██████████| 143/143 [03:08<00:00,  1.32s/it]


### Remove unnecessary files

`csvt` files are not needed for this project (they are used in QGIS).

In [20]:
os.system("rm data/csv/*.csvt")

0

Detailed list of tables and their relations can be found [here](https://www.bdnb.eu/schema/v07_2022_10_c/index.html).

In [ ]:
unrelevant_tables = [
    'proprietaire', 'parcelle', 'dvf_open', 'merimee', 'qpv',
    'dvf_open_statistique', 'dvf_open_representatif', 'argiles',
    'geospx', 'radon', 'osm_building'
]

for table in unrelevant_tables:
    os.system(f"rm data/csv/{table}.csv")
    os.system(f"rm data/csv/batiment_groupe_{table}.csv")
    os.system(f"rm data/csv/rel_batiment_groupe_{table}.csv")
    
print(f"Number of files in the data folder: {len(os.listdir('data/csv'))}")

In [3]:
df_bats = pd.read_csv("data/csv/batiment_groupe.csv", sep=",")

In [25]:
print(f"Number of rows in the df_bats dataframe: {df_bats.shape[0]}")

Number of rows in the df_bats dataframe: 71458


### Merging Buildings with their DPE

In [4]:
df_dpe = pd.read_csv("data/csv/batiment_groupe_dpe.csv", sep=",")
df_dpe = df_dpe[["batiment_groupe_id", "class_conso_ener_mean"]]
df_bats = df_bats.merge(df_dpe, on="batiment_groupe_id", how="right")
print (f"Number of rows in the df_bats dataframe after the merge: {df_bats.shape[0]}")

Number of rows in the df_bats dataframe after the merge: 47087


In [5]:
df_ffo = pd.read_csv("data/csv/batiment_groupe_ffo_bat.csv", sep=",")
df_bats = df_bats.merge(df_ffo, on="batiment_groupe_id", how="left")

In [6]:
df_conso_elec = pd.read_csv("data/csv/batiment_groupe_dle_elec_2020.csv", sep=",")
df_bats = df_bats.merge(df_conso_elec, on="batiment_groupe_id", how="left")

In [56]:
df_bats.head()

,geom_groupe,batiment_groupe_id,code_departement_insee_x,s_geom_groupe,code_iris,code_commune_insee,libelle_commune_insee,code_epci_insee,contient_fictive_geom_groupe,class_conso_ener_mean,code_departement_insee_y,annee_construction,usage_niveau_1_txt,mat_mur_txt,mat_toit_txt,nb_log,code_departement_insee,nb_pdl_res,nb_pdl_pro,nb_pdl_tot,conso_res,conso_pro,conso_tot,conso_res_par_pdl,conso_pro_par_pdl,conso_tot_par_pdl
0,MULTIPOLYGON (((650293.046070325 6860715.43352...,75106000AX0072_8f312a91b4c6c1f,75,NaN,751062306.0,75056.0,Paris,200054781.0,1,G,75,1890.0,Résidentiel collectif,MEULIERE - PIERRE,ZINC ALUMINIUM,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MULTIPOLYGON (((655378.647803063 6864050.06480...,75120000AE0076_0f572212c6d8bef,75,NaN,751207712.0,75056.0,Paris,200054781.0,1,E,75,2008.0,Résidentiel collectif,INDETERMINE,INDETERMINE,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"MULTIPOLYGON (((651451.4 6863131.9,651446.5 68...",75101000AV0068_974bcbe7e818135,75,229.0,751010301.0,75056.0,Paris,200054781.0,0,N,75,1770.0,Résidentiel collectif,BOIS - MEULIERE,ARDOISES - ZINC ALUMINIUM,11,75.0,11.0,0.0,11.0,41414.8,0.0,41414.8,3764.982,NaN,3764.982
3,"MULTIPOLYGON (((652719.9 6863356.4,652725.8 68...",75103000AB0044_5dce797b740e30f,75,54.0,751030901.0,75056.0,Paris,200054781.0,0,G,75,1800.0,Résidentiel collectif,PIERRE - AUTRES,TUILES,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MULTIPOLYGON (((648013.005160536 6861418.02680...,75115000DK0052_1fa8850f144d769,75,NaN,751155920.0,75056.0,Paris,200054781.0,1,N,75,1900.0,Résidentiel collectif,BRIQUES,TUILES,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Prepare data for training

In [7]:
df_bats = df_bats.dropna(axis=1, thresh=0.5*df_bats.shape[0])
df_bats = df_bats.dropna(axis=0)

In [8]:
y = df_bats["class_conso_ener_mean"]
X = df_bats.drop(["class_conso_ener_mean"], axis=1)

In [181]:
X.shape, y.shape

((32101, 24), (32101,))

In [9]:
features_to_keep = [
    'annee_construction', 'usage_niveau_1_txt', 'mat_mur_txt', 'mat_toit_txt', 'nb_log'
]
X = X[features_to_keep]

In [10]:
X.shape

(32101, 5)

In [11]:
categorical_features = [
    'usage_niveau_1_txt', 'mat_mur_txt', 'mat_toit_txt'
]
ohe = OneHotEncoder(sparse=False)
X_ohe = ohe.fit_transform(X[categorical_features])
X_ohe = pd.DataFrame(X_ohe, columns=ohe.get_feature_names_out(categorical_features), index=X.index)

/home/simon/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [14]:
X = pd.concat([X.drop(categorical_features, axis=1), X_ohe], axis=1)
X.shape

(32101, 51)

In [19]:
le = LabelEncoder()
y_encoded = le.fit_transform(y)

array([7, 5, 5, ..., 3, 3, 4])

### Split data into train and test sets

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

### Train model

In [30]:
xgb_model = xgb.XGBClassifier(objective="multi:softmax", random_state=42)
grid_params = {
    "n_estimators": [20, 50, 60],
    "max_depth": [1, 2, 3],
    "learning_rate": [0.001, 0.01, 0.1]
}

search = GridSearchCV(xgb_model, grid_params, scoring="accuracy", cv=5, n_jobs=-1)

In [31]:
search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     gamma=None, gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_bin=None,
                                     max_ca...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None,
                                     objective='multi:softmax', predictor=None,
                                     random_state=42, reg_alpha=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.001, 0.01, 0.1],
                         'max_depth': [1, 2, 3], 'n_estimators': [20, 50, 60]},
             scoring='accuracy')

In [27]:
search.best_params_

{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50}

In [28]:
search.best_score_

0.35623052959501555